## Import des fichiers et bibliothèques

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, log_loss, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score, train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from joblib import dump
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet('df_breast_cancer.parquet')

Création de la liste des modèles

In [3]:
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    NuSVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    HistGradientBoostingClassifier(),
    LinearSVC(),
    MLPClassifier(),
    XGBClassifier(),
    LGBMClassifier(verbose=-1)]

In [5]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']

Création de la liste des scalers

In [6]:
scalers = [
    ("Unscaled data", X),
    ("Data after standard scaling", StandardScaler().fit_transform(X)),
    ("Data after min-max scaling", MinMaxScaler().fit_transform(X)),
    ("Data after max-abs scaling", MaxAbsScaler().fit_transform(X)),
    (
        "Data after robust scaling",
        RobustScaler(quantile_range=(25, 75)).fit_transform(X),
    ),
    (
        "Data after power transformation (Yeo-Johnson)",
        PowerTransformer(method="yeo-johnson").fit_transform(X),
    ),
    #(
    #    "Data after power transformation (Box-Cox)",
    #    PowerTransformer(method="box-cox").fit_transform(X),
    #),
    (
        "Data after quantile transformation (uniform pdf)",
        QuantileTransformer(
            output_distribution="uniform", random_state=42
        ).fit_transform(X),
    ),
    (
        "Data after quantile transformation (gaussian pdf)",
        QuantileTransformer(
            output_distribution="normal", random_state=42
        ).fit_transform(X),
    ),
    ("Data after sample-wise L2 normalizing", Normalizer().fit_transform(X)),
]

c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\sklearn\preprocessing\_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\sklearn\preprocessing\_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(


## Comparaison des différents scalers avec les différents modèles

In [7]:
# Logging for Visual Comparison
log_cols=["Scaler","Classifier", "Accuracy", "Recall", "Log Loss"]
log = pd.DataFrame(columns=log_cols)
for X in scalers :
    for clf in classifiers:
        scaler = X[0]
        X_train, X_test, y_train, y_test = train_test_split(X[1], y, random_state=42)
        clf.fit(X_train, y_train)
        name = clf.__class__.__name__

        #print("="*30)
        #print(name)

        #print('****Results****')
        train_predictions = clf.predict(X_test)
        acc = accuracy_score(y_test, train_predictions)
        rec = recall_score(y_test, train_predictions)
        ll = log_loss(y_test, train_predictions)
        #print("Accuracy: {:.4%}".format(acc))
        try:
            train_predictions = clf.predict_proba(X_test)
            
            #print("Log Loss: {}".format(ll))
        except:
            pass
        log_entry = pd.DataFrame([[scaler, name, acc*100, rec*100, ll]], columns=log_cols)
        log = pd.concat([log,log_entry], ignore_index=True)

C:\Users\Bamdantag\AppData\Local\Temp\ipykernel_3572\3163831575.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  log = pd.concat([log,log_entry], ignore_index=True)
c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Bamdantag\miniconda3\envs\brandnew\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` wil

In [8]:
log.sort_values(['Accuracy','Recall','Log Loss'],ascending=[False, False, True]).head(50)

,Scaler,Classifier,Accuracy,Recall,Log Loss
87,Data after power transformation (Yeo-Johnson),MLPClassifier,96.402878,96.969697,1.296534
109,Data after quantile transformation (gaussian pdf),RandomForestClassifier,96.402878,93.939394,1.296534
27,Data after standard scaling,MLPClassifier,95.683453,95.454545,1.555841
10,Unscaled data,HistGradientBoostingClassifier,95.683453,92.424242,1.555841
13,Unscaled data,XGBClassifier,95.683453,92.424242,1.555841
25,Data after standard scaling,HistGradientBoostingClassifier,95.683453,92.424242,1.555841
28,Data after standard scaling,XGBClassifier,95.683453,92.424242,1.555841
40,Data after min-max scaling,HistGradientBoostingClassifier,95.683453,92.424242,1.555841
43,Data after min-max scaling,XGBClassifier,95.683453,92.424242,1.555841
55,Data after max-abs scaling,HistGradientBoostingClassifier,95.683453,92.424242,1.555841


## Création du jeu de test et du jeu d'entrainement ainsi que suréchantillonage et mise à l'échelle

In [10]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ros)
X_test_scaled = scaler.transform(X_test)

## Comparaison des scores de nos algorithmes selectionnés et création d'un voting classifier

In [16]:
skf = StratifiedKFold(n_splits=5)
results = {
    'models' : [
        ('hgbc', HistGradientBoostingClassifier()),
        ('nsvc', NuSVC(probability=True)),
        ('mlpc', MLPClassifier()),
        ('lsvc', SVC(kernel='linear',probability=True)),
        ('rfc', RandomForestClassifier()),
        ('xgbc', XGBClassifier()),
        ],
    'mean_score' :[],
    'std_dev_score' :[]
}
for _, model in results.get('models') :
    scores = cross_val_score(model,
                            X_test_scaled,
                            y_test,
                            scoring = 'roc_auc',
                            cv = skf,
                            n_jobs = -1)
    results['mean_score'].append(scores.mean())
    results['std_dev_score'].append(scores.std())

    name = type(model).__name__
    print(f'{name} - Roc AUC score: {scores.mean():.4f} ± {scores.std():.4f}')

HistGradientBoostingClassifier - Roc AUC score: 0.9690 ± 0.0219
NuSVC - Roc AUC score: 0.9698 ± 0.0248
MLPClassifier - Roc AUC score: 0.9804 ± 0.0166
SVC - Roc AUC score: 0.9804 ± 0.0153
RandomForestClassifier - Roc AUC score: 0.9699 ± 0.0245
XGBClassifier - Roc AUC score: 0.9732 ± 0.0141


In [17]:
voting = VotingClassifier(
    estimators=results.get('models'),
    voting='soft',
    weights=results.get('mean_score'),
    verbose=True,
    n_jobs=-1

)

In [18]:
voting.fit(X_train_scaled,y_train_ros)

VotingClassifier(estimators=[('hgbc', HistGradientBoostingClassifier()),
                             ('nsvc', NuSVC(probability=True)),
                             ('mlpc', MLPClassifier()),
                             ('lsvc', SVC(kernel='linear', probability=True)),
                             ('rfc', RandomForestClassifier()),
                             ('xgbc',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=N...
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))],
                 n_jobs=-1, verbose=True, voting='soft',
                 weights=[0.9690214547357405, 0.9698273155416013,
                          0.980376766091052, 0.9803872318158033,
                          0.9699005756148613, 0.9732025117739405])

In [19]:
predictions = voting.predict(X_test_scaled)

Scoring puis export

In [20]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print("\n")
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97        73
           1       0.98      0.94      0.96        66

    accuracy                           0.96       139
   macro avg       0.97      0.96      0.96       139
weighted avg       0.96      0.96      0.96       139



[[72  1]
 [ 4 62]]


In [22]:
dump(scaler, 'scaler_cancer.joblib')
dump(voting, 'voting_cancer.joblib')

['voting_cancer.joblib']